<a href="https://colab.research.google.com/github/mhtattersall/mhtattersall/blob/main/FinanceKPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necesary libraries
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# Download historical data for required stocks
tickers = ["AMZN","GOOG","MSFT"] #list of tickers
ohlcv_data = {} #new dictionary

# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='7mo',interval='1d')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
print(ohlcv_data["AMZN"]) #print df for a stock

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-04-10  100.959999  102.199997   99.570000  102.169998  102.169998   
2023-04-11  100.800003  101.000000   99.010002   99.919998   99.919998   
2023-04-12  100.400002  100.510002   97.709999   97.830002   97.830002   
2023-04-13   98.949997  102.570000   98.709999  102.400002  102.400002   
2023-04-14  102.070000  103.199997  101.110001  102.510002  102.510002   
...                ...         ...         ...         ...         ...   
2023-11-03  138.990005  139.490005  137.449997  138.600006  138.600006   
2023-11-06  138.759995  140.729996  138.360001  139.740005  139.740005   
2023-11-07  140.550003  143.369995  140.500000  142.710007  142.710007   
2023-11-08  142.970001  143.119995  141.220001  142.080002  142.080002   
2023-11-09  142.020004  142.649994  139.839996  140.600006  140.600006   

              Volume  
Date          

In [4]:
def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df = DF.copy() #copy the df in the dictionary
    df["return"] = DF["Adj Close"].pct_change() #new column with percentage change function of pandas applied to each day (or period)
    df["cum_return"] = (1 + df["return"]).cumprod() #new column with 1+daily return, cumulative product return function of pandas to each daily return figure
    n = len(df)/252 #length of the df divided by trading days in the year #if hourly data it would be 252/8hrs
    CAGR = ((df["cum_return"][-1])**(1/n) - 1) #CAGR formula #take the last row in the cum-return column
    return CAGR

for ticker in ohlcv_data:
    print("CAGR of {} = {}".format(ticker,CAGR(ohlcv_data[ticker]))) #apply function to each stock #these need to be *100 for percentages

CAGR of AMZN = 0.7098266529241155
CAGR of GOOG = 0.4184858538553218
CAGR of MSFT = 0.4582411386224692


In [5]:
def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy() #copy the df in the dictionary
    df["daily_ret"] = DF["Adj Close"].pct_change() #new column with percentage change pandas function applied to each day (or period)
    vol = df["daily_ret"].std() * np.sqrt(252) #1+daily return, standard deviation function of pandas with daily returns, numpy square root function
    return vol

for ticker in ohlcv_data:
    print("vol for {} = {}".format(ticker,volatility(ohlcv_data[ticker]))) #apply function to each stock

vol for AMZN = 0.3321837474043249
vol for GOOG = 0.28008134660601225
vol for MSFT = 0.24372485517824746


In [6]:
def sharpe(DF, rf): #pass df of the stock and a risk-free rate
    "function to calculate Sharpe Ratio of a trading strategy"
    df = DF.copy() #copy the df in the dictionary
    return (CAGR(df) - rf)/volatility(df) #apply Sharpe Ratio formula

In [7]:
def sortino(DF, rf):
    "function to calculate Sortino Ratio of a trading strategy"
    df = DF.copy() #copy the df in the dictionary
    df["return"] = df["Adj Close"].pct_change() #new column with percentage change pandas function applied to each day (or period)
    neg_return = np.where(df["return"]>0,0,df["return"]) #keep negative returns, else return them as 0. #returned as an array
    #below you will see two ways to calculate the denominator (neg_vol), some people use the
    #standard deviation of negative returns while others use a downward deviation approach,
    #you can use either. However, downward deviation approach is more widely used
    neg_vol = np.sqrt((pd.Series(neg_return[neg_return != 0]) ** 2).mean() * 252) #downward deviation #filter out zero values #convert to a pandas series #
    #neg_vol = pd.Series(neg_return[neg_return != 0]).std() * np.sqrt(252) #standard deviation of negative returns
    return (CAGR(df) - rf)/neg_vol #apply Sortino Ratio formula

In [8]:
for ticker in ohlcv_data:
    print("Sharpe of {} = {}".format(ticker,sharpe(ohlcv_data[ticker],0.03))) #assume risk-free rate 3%
    print("Sortino of {} = {}".format(ticker,sortino(ohlcv_data[ticker],0.03))) #assume risk-free rate 3%

Sharpe of AMZN = 2.0465379725415924
Sortino of AMZN = 2.134662961945856
Sharpe of GOOG = 1.3870465083196029
Sortino of GOOG = 1.3545444443012689
Sharpe of MSFT = 1.757067978599377
Sortino of MSFT = 1.8826096843902393


In [9]:
def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy() #copy the df in the dictionary
    df["return"] = df["Adj Close"].pct_change() #new column with percentage change function of pandas applied to each day (or period)
    df["cum_return"] = (1+df["return"]).cumprod() #new column with 1+daily return, cumulative product return function of pandas to each daily return figure
    df["cum_roll_max"] = df["cum_return"].cummax() #new column with cumulative max function appled to cum-return
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"] #new column with the cum_roll_max minus the cum_return at that point
    return (df["drawdown"]/df["cum_roll_max"]).max() #return the maximum drawdown expressed as a ratio to the cum_roll_max

In [10]:
def calmar(DF):
    "function to calculate calmar ratio"
    df = DF.copy() #copy the df in the dictionary
    return CAGR(df)/max_dd(df) #apply Calmar Ratio formula

In [11]:
for ticker in ohlcv_data:
    print("max drawdown of {} = {}".format(ticker,max_dd(ohlcv_data[ticker])))
    print("calmar ratio of {} = {}".format(ticker,calmar(ohlcv_data[ticker])))

max drawdown of AMZN = 0.17452540796322294
calmar ratio of AMZN = 4.067182315790343
max drawdown of GOOG = 0.1291460537507817
calmar ratio of GOOG = 3.240407598228984
max drawdown of MSFT = 0.12987593237872266
calmar ratio of MSFT = 3.5282991254008658
